In [ ]:
# import torch
# import signatory
# import numpy as np
# import itertools
# import trading_strategy
# import utils
# import esig # to deal with indexing of signature

depth = 3
delta = 0.3
ts = trading_strategy.TradingStrategy(depth, delta)
batch_size = 2
d = 3 # number of stocks
N = 1 # size of info
Z_dim = N+d+1 # dimension of Z
T = 7 # number of time steps
batch_stock_data = torch.rand(batch_size, T, d) # X
batch_info_data = torch.rand(batch_size, T, N) # f
ts.fit(batch_stock_data, batch_info_data)

In [17]:
import itertools

def word_to_i(word, d):
    """
    Given a word written in the alphabet {0, 1, ..., d-1}, return its index in the lexicographic order (basically).
    """
    k = len(word) # we're accessing the k-th signature term
    s = sum(d**i for i in range(k)) # 1 + d + d^2 + ... + d^(k-1)
    c = 0
    for i, w in enumerate(word):
        c += int(w) * d**(k-1-i)
    return s + c

d = 3 # dimension
N = 3 # max word length
words = []
for length in range(N+1):
    alphabet = [str(i) for i in range(d)]
    words += [''.join(seq) for seq in itertools.product(alphabet, repeat=length)]

for word in words:
    i = word_to_i(word, d)
    print(f'word = {word}, i = {i}')

word = , i = 0
word = 0, i = 1
word = 1, i = 2
word = 2, i = 3
word = 00, i = 4
word = 01, i = 5
word = 02, i = 6
word = 10, i = 7
word = 11, i = 8
word = 12, i = 9
word = 20, i = 10
word = 21, i = 11
word = 22, i = 12
word = 000, i = 13
word = 001, i = 14
word = 002, i = 15
word = 010, i = 16
word = 011, i = 17
word = 012, i = 18
word = 020, i = 19
word = 021, i = 20
word = 022, i = 21
word = 100, i = 22
word = 101, i = 23
word = 102, i = 24
word = 110, i = 25
word = 111, i = 26
word = 112, i = 27
word = 120, i = 28
word = 121, i = 29
word = 122, i = 30
word = 200, i = 31
word = 201, i = 32
word = 202, i = 33
word = 210, i = 34
word = 211, i = 35
word = 212, i = 36
word = 220, i = 37
word = 221, i = 38
word = 222, i = 39
